## [Analysis of Medicare Dialysis Facilities data from 2017 to 2020](https://data.cms.gov/quality-of-care/medicare-dialysis-facilities)

The purpose of this notebook is to ingest data from the Center for Medicare and Medicaid Services of dialysis facilities for 2017 through 2020 and answer research questions about characteristics of End-Stage Renal Disease (ESRD) patients, medicare dialysis facilities, and standardized readmission rates (SRR), an indicator of quality of care in these facilities.

The key research questions to answer are:
* Has the number of patients in dialysis facilities increased between 2017 and 2020?
* Is there an association between the number of patients in dialysis facilities and the share of patients sharing from comorbidities such as diabetes?
* Do patients who received Pre-ESRD Nephrologist Care, on average, have lower SRR compared to those that did not?

In [0]:
# The data for the analysis is saved in an AWS S3 object and is ingested below.
df = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load("s3://medicare-dialysis-facilities/Medicare_Dialysis_Facility_Data_2022.csv")

# All features are string type except the CMS Certification Number (CCN), ESRD Network, and the measure value or score of every facility. 

In [0]:
# Display the first five rows of the data frame.
df.limit(5).toPandas()

,state,CCN,Provider_Name,city,Ownership_Type,ESRD_Network,NPI,Chain,Modality,Alternate_CCNs,Measure,Measure_Score,year,Measure_ID
0,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: COVID - Number of Medicare dialysis patient...,12.0,"Quarter 2, 2020",allmcpatq2_f
1,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: COVID - Number of patients first identified...,0.0,"Quarter 2, 2020",allmcFcovpatq2_f
2,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: COVID - % of Medicare patients first identi...,0.0,"Quarter 2, 2020",allmcFcovpatPq2_f
3,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: COVID - Number of patients ever identified ...,0.0,"Quarter 2, 2020",allmcEcovpatq2_f
4,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: COVID - % of Medicare patients ever identif...,0.0,"Quarter 2, 2020",allmcEcovpatPq2_f


In [0]:
# Understand the data more by determining the shape of the data frame
# The data frame contains 10,821,385 rows and 14 columns
df.count(), len(df.columns)

Out[21]: (10821385, 14)

In [0]:
# Most of the dialysis facilities are for-profit institutions.
df.groupBy('Ownership_Type').count().show()

+--------------+-------+
|Ownership_Type|  count|
+--------------+-------+
|    For Profit|9624215|
|    Non-profit|1193248|
|   Unavailable|   3921|
|          null|      1|
+--------------+-------+



In [0]:
# The most common modality of dialysis is hemodialysis and peritoneal dialysis and the least frequent type of peritoneal dialysis
df.groupBy('Modality').count().sort("count").toPandas()

,Modality,count
0,None,1
1,Unavailable,88161
2,Peritoneal Dialysis,144120
3,Hemodialysis,4810913
4,Hemodialysis and Peritoneal Dialysis,5778190


In [0]:
# The following are the tabulations of the `year` feature. Some measures were reported by quarter in 2020 and therefore, have data listed separately from the 2020 year value.
df.groupBy('year').count().sort("count").toPandas()

,year,count
0,None,1
1,n/a,54619
2,"Quarter 4, 2020",87857
3,"Quarter 3, 2020",90557
4,"Quarter 2, 2020",92199
5,"Quarter 1, 2020",97890
6,2017-19,113469
7,2017-20,521687
8,2020,2286892
9,2017,2421398


In [0]:
# Examine describe statistics of key variables

In [0]:
# Create a temporary view of the data for writing SQL queries
# query the data below.
df.createOrReplaceTempView("medicare_dialysis_facilities")

In [0]:
%sql

select * from `medicare_dialysis_facilities` LIMIT 5;

state,CCN,Provider_Name,city,Ownership_Type,ESRD_Network,NPI,Chain,Modality,Alternate_CCNs,Measure,Measure_Score,year,Measure_ID
AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300","F: COVID - Number of Medicare dialysis patients, 2020-Q2",12.0,"Quarter 2, 2020",allmcpatq2_f
AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300","F: COVID - Number of patients first identified with COVID, 2020-Q2",0.0,"Quarter 2, 2020",allmcFcovpatq2_f
AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300","F: COVID - % of Medicare patients first identified with COVID, 2020-Q2",0.0,"Quarter 2, 2020",allmcFcovpatPq2_f
AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300","F: COVID - Number of patients ever identified with COVID (prior to or during current quarter), 2020-Q2",0.0,"Quarter 2, 2020",allmcEcovpatq2_f
AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300","F: COVID - % of Medicare patients ever identified with COVID, 2020-Q2",0.0,"Quarter 2, 2020",allmcEcovpatPq2_f


In [0]:
%sql

SELECT Measure,
       ROUND(COUNT(Measure_Score),0) AS cnt_value,
       ROUND(AVG(Measure_Score),1) AS avg_value,
       ROUND(MIN(Measure_Score),1) AS min_value,
       ROUND(MAX(Measure_Score),1) AS max_value,
       ROUND(STD(Measure_Score),1) AS std_value
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("allcnty1_f","allcnty2_f","allcnty3_f","allcnty4_f",
         "dbprimy1_f","dbprimy2_f","dbprimy3_f","dbprimy4_f",
         "gnprimy1_f","gnprimy2_f","gnprimy3_f","gnprimy4_f",
         "htprimy1_f","htprimy2_f","htprimy3_f","htprimy4_f",
         "nephy12my1_f","nephy12my2_f","nephy12my3_f","nephy12my4_f",
         "srry1_f","srry2_f","srry3_f","srry4_f")
GROUP BY Measure
ORDER BY Measure DESC;

Measure,cnt_value,avg_value,min_value,max_value,std_value
"F: SRR - Standardized Readmission Ratio, 2020",6241,1.0,0.0,2.9,0.4
"F: SRR - Standardized Readmission Ratio, 2019",7169,1.0,0.0,2.5,0.3
"F: SRR - Standardized Readmission Ratio, 2018",6893,1.0,0.0,3.8,0.3
"F: SRR - Standardized Readmission Ratio, 2017",6632,1.0,0.0,2.3,0.3
"F (AFS): Facility Information - Facility Information - Number of patients, 2020",7796,100.6,0.0,1561.0,66.2
"F (AFS): Facility Information - Facility Information - Number of patients, 2019",7744,101.1,0.0,1400.0,67.4
"F (AFS): Facility Information - Facility Information - Number of patients, 2018",7513,101.0,0.0,1388.0,67.9
"F (AFS): Facility Information - Facility Information - Number of patients, 2017",7165,103.1,0.0,1455.0,68.7
"F (2728): Incident Patients - Pre-ESRD Nephrologist Care Greater Than 12 months, 2020",3021,33.1,0.0,100.0,23.7
"F (2728): Incident Patients - Pre-ESRD Nephrologist Care Greater Than 12 months, 2019",4789,32.0,0.0,100.0,22.8


Overall, it appears that on average the SRR has not changed significantly between 2017 and 2020. However, the average SRR among all facilities has increased from 2019 but was lower compared to the highest on 2018. The SRR is a measure of quality because it evaluates total observed readmissions associated with a facility to the expected readmissions given the characteristics of the patients served in that facility. A lower SRR is evidence that fewer readmissions were necessary compared to the expected number and may signal that the quality of care was sufficiently high as to not require readmission into the facility or hospital. However, additional contextual factors such as patient comorbidities and care taken after a dialysis procedure may influence the rate at which patients are readmitted. The number of patients, on average, has remained at around 100. On average, the share of incident patients who received pre-ESRD nephrologist care for more than 12 months in creased from 31% in 2017 to 33% in 2020, and on average the share or incident patients who reported diabetes as the primary cause of ESRD increased from 48.7% in 2017 to 49% in 2020.

In [0]:
# Determine the number of null values in the dataframe.

from pyspark.sql.functions import *
display(df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]))

# This resource was used in identifying the number of null values in each column of the data frame: https://www.projectpro.io/recipes/get-null-count-of-each-column-of-dataframe-pyspark-databricks

state,CCN,Provider_Name,city,Ownership_Type,ESRD_Network,NPI,Chain,Modality,Alternate_CCNs,Measure,Measure_Score,year,Measure_ID
1,1,1,1,1,1,1,1,1,1,1,1,1,1


The map below shows the states with the highest number of dialysis patients, including California, Texas, Florida, and New York. These are some of the most populated states, therefore, it may be expected that overall they will have a higher number of patients. The count of patients can be compared to the population size for the age group of dialysis patients in the dataset, which typically includes individuals above age 60.

In the following cell, the percentage growth in the number of dialysis patients by state is computed. States such as Utah, Maine, South Dakota, and Arizona account for the four highest states with the largest growth in the number of dialysis patients.

In response to the first research question of growth in the number of patients, on average, the number of patients is stable at around 100, in 2017 and 2020. However, some states have significantly higher growth rates in the number of dialysis patients compared to other states. It is imperative for state health agencies to understand the reasons behind the growth in the number of dialysis patients and implement preventative policies that will decrease the risk of ESRD for elderly patients.

In [0]:
%sql

SELECT state,
       SUM(Measure_Score) AS `Number of patients in 2020`
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("allcnty4_f") AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')
GROUP BY state
ORDER BY `Number of patients in 2020` DESC;

state,Number of patients in 2020
CA,104732.0
TX,79563.0
FL,52368.0
NY,45855.0
IL,32422.0
GA,32416.0
PA,28314.0
OH,27780.0
NC,27472.0
MI,22712.0


In [0]:
%sql

WITH patients_2017(state,nbr_patients_17)
AS (SELECT state,
       SUM(Measure_Score) AS `nbr_patients`
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("allcnty1_f") AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')
GROUP BY state
ORDER BY `nbr_patients` DESC),

patients_2020(state,nbr_patients_20)
AS (SELECT state,
       SUM(Measure_Score) AS `nbr_patients`
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("allcnty4_f") AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')
GROUP BY state
ORDER BY `nbr_patients` DESC)

SELECT a.state,
       a.nbr_patients_17 AS `Number of patients in 2017`,
       b.nbr_patients_20 AS `Number of patients in 2020`,
       ROUND(100 * ((b.nbr_patients_20 - a.nbr_patients_17) / (a.nbr_patients_17)),1) AS `Percentage growth`
FROM patients_2017 AS a
  FULL JOIN patients_2020 AS b
    ON a.state = b.state
ORDER BY `Percentage growth` DESC
LIMIT 10;

state,Number of patients in 2017,Number of patients in 2020,Percentage growth
UT,2910.0,3511.0,20.7
ME,1541.0,1808.0,17.3
SD,1500.0,1734.0,15.6
AZ,14695.0,16781.0,14.2
AK,817.0,931.0,14.0
NV,6714.0,7605.0,13.3
HI,5034.0,5691.0,13.1
NM,4889.0,5453.0,11.5
AR,6510.0,7227.0,11.0
NC,24813.0,27472.0,10.7


Based on the scatter plot below, it appears that there is a moderately negative relationship between the growth rate in the average share of patients whose primary cause of ESRD is diabetes and the growth in the number of patients. In other words, on average, in states where the average growth in the share of incidence patients whose primary cause of ESRD is diabetes, the growth rate in the number of patients tends to be lower. This highlights that there may be more reasons that explain the growth in the number of dialysis patient beyond diabetes as a comorbidity.

In [0]:
%sql

WITH patients_2017(state,nbr_patients_17)
AS (SELECT state,
       SUM(Measure_Score) AS `nbr_patients`
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("allcnty1_f") AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')
GROUP BY state
ORDER BY `nbr_patients` DESC),

patients_2020(state,nbr_patients_20)
AS (SELECT state,
       SUM(Measure_Score) AS `nbr_patients`
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("allcnty4_f") AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')
GROUP BY state
ORDER BY `nbr_patients` DESC),

diabetes_2020(state,diabetes_20)
AS (SELECT state,
       AVG(Measure_Score) AS `diabetes_20`
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("dbprimy4_f") AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')
GROUP BY state
ORDER BY `diabetes_20` DESC),

diabetes_2017(state,diabetes_17)
AS (SELECT state,
       AVG(Measure_Score) AS `diabetes_17`
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("dbprimy1_f") AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')
GROUP BY state
ORDER BY `diabetes_17` DESC)

SELECT a.state,
       a.nbr_patients_17 AS `Number of patients in 2017`,
       b.nbr_patients_20 AS `Number of patients in 2020`,
       ROUND(100 * ((b.nbr_patients_20 - a.nbr_patients_17) / (a.nbr_patients_17)),1) AS `% growth in patients`,
       c.diabetes_17 AS `Average share of patients whose primary cause of ESRD is diabetes in 2017`,
       d.diabetes_20 AS `Average share of patients whose primary cause of ESRD is diabetes in 2020`,
       ROUND(100 * ((d.diabetes_20 - c.diabetes_17) / (c.diabetes_17)),1) AS `% growth in diabetes as primary cause of ESRD`      
FROM patients_2017 AS a
  FULL JOIN patients_2020 AS b
    ON a.state = b.state
  FULL JOIN diabetes_2017 AS c
    ON a.state = c.state
  FULL JOIN diabetes_2020 AS d
    ON a.state = d.state
ORDER BY `% growth in patients` DESC;

state,Number of patients in 2017,Number of patients in 2020,% growth in patients,Average share of patients whose primary cause of ESRD is diabetes in 2017,Average share of patients whose primary cause of ESRD is diabetes in 2020,% growth in diabetes as primary cause of ESRD
UT,2910.0,3511.0,20.7,50.97444444444446,62.135454545454536,21.9
ME,1541.0,1808.0,17.3,50.40555555555556,55.728333333333325,10.6
SD,1500.0,1734.0,15.6,63.62125,47.9125,-24.7
AZ,14695.0,16781.0,14.2,57.41348314606743,58.169384615384615,1.3
AK,817.0,931.0,14.0,59.512,47.519999999999996,-20.2
NV,6714.0,7605.0,13.3,50.69926829268294,42.167500000000004,-16.8
HI,5034.0,5691.0,13.1,69.66238095238096,63.96421052631579,-8.2
NM,4889.0,5453.0,11.5,63.489696969696965,61.719047619047615,-2.8
AR,6510.0,7227.0,11.0,45.59276595744681,42.919090909090905,-5.9
NC,24813.0,27472.0,10.7,49.26986394557822,49.27642201834861,0.0


In [0]:
%sql

SELECT state,
       CCN,
       Provider_Name,
       Measure_Score
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("srry4_f") 
AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')
LIMIT 5;

state,CCN,Provider_Name,Measure_Score
AL,12500,FMC CAPITOL CITY,0.55
AL,12501,GADSDEN DIALYSIS,1.14
AL,12502,TUSCALOOSA UNIVERSITY DIALYSIS,1.24
AL,12505,PHYSICIANS CHOICE DIALYSIS-MONTGOMERY,1.03
AL,12506,DOTHAN DIALYSIS,1.25


In [0]:
%sql

WITH pre_esrd(state,CCN,Provider_Name,Measure_Score,`Received pre-ESRD care for 12+ months`)
AS (SELECT state,
       CCN,
       Provider_Name,
       Measure_Score,
       (CASE WHEN Measure_Score > 60 THEN "Yes" ELSE "No" END) AS `Received pre-ESRD care for 12+ months`
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("nephy12my4_f") 
AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI')),

srr(state,CCN,Provider_Name,SRR)
AS (SELECT state,
       CCN,
       Provider_Name,
       Measure_Score
FROM `medicare_dialysis_facilities`
WHERE Measure_ID IN("srry4_f") 
AND state NOT IN('AS','FM','GU','MH','MP','PR','PW','VI'))

SELECT a.`Received pre-ESRD care for 12+ months` AS `60+% of patients received pre-ESRD care`,
       a.state AS `State`,
       AVG(b.SRR) AS `Average Standardized Readmission Ratio`
FROM pre_esrd AS a
  FULL JOIN srr AS b
    ON a.CCN = b.CCN
WHERE a.`Received pre-ESRD care for 12+ months` IS NOT NULL
GROUP BY a.state,a.`Received pre-ESRD care for 12+ months`
ORDER BY state,`60+% of patients received pre-ESRD care`;

60+% of patients received pre-ESRD care,State,Average Standardized Readmission Ratio
No,AK,0.8175
No,AL,1.0221052631578948
Yes,AL,0.86
No,AR,0.9012903225806452
Yes,AR,1.09
No,AZ,1.0226315789473683
Yes,AZ,0.9762500000000001
No,CA,1.0193641618497107
Yes,CA,1.1035
No,CO,0.8925


As the chart above demonstrates, for 2020, on average, facilities for which 60% or more of patients received pre-ESRD nephrologist care for more than 12 months, had lower standardized readmission rates in states such as New Mexico. This is expected, as greater availability of specialized care prior and during the process of dialysis is expected to result in patients being better prepared to manage their condition and not have to be readmitted to a hospital or dialysis facility due to a dialysis procedure. However, in other states such as Mississippi, the SRR is significantly higher for patients who received more than 12 months of pre-ESRD care. Therefore, state health agencies can learn about their best practices and programs in states such as New Mexico, which have on average, lower readmission rates, compared to other states, particularly from facilities whose patients received 12 or more months of pre-ESRD nephrologist care.

The `Overview` notebook from Databricks was used as a reference in the initial steps of this analysis, particularly in the data ingestion commands and the creation of a temporary view for querying the dataset.